이미 만들어 둔, 문서 - 단어 행렬을 이용하여 NMF 를 이용한 토픽모델링을 합니다.

In [1]:
import config
from lovit_textmining_dataset.navernews_10days import get_bow

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')

soynlp=0.0.49
added lovit_textmining_dataset


300,91 개의 문서가 9,774 개의 단어로 표현되어 있습니다.

In [2]:
x.shape

(30091, 9774)

사용법은 sklearn.decomposition.TruncatedSVD 와 같습니다.

하지만 학습 시간이 조금 긴 편입니다. SVD 가 16.2 초 만에 학습되었지만, NMF 는 5 분의 학습 시간이 필요합니다. 학습이 오래 걸리니 한 번 학습한 뒤, 파일을 저장해둡니다.

In [3]:
%%time

import pickle
from sklearn.decomposition import NMF

TRAIN = False
if TRAIN:
    nmf = NMF(n_components=100)
    y = nmf.fit_transform(x)

    with open('./2016-10-20-nmf.pkl', 'wb') as f:
        pickle.dump(nmf, f)
    with open('./2016-10-20-nmf_y.pkl', 'wb') as f:
        pickle.dump(y, f)

else:
    # load trained model
    with open('./2016-10-20-nmf.pkl', 'rb') as f:
        nmf = pickle.load(f)
    with open('./2016-10-20-nmf_y.pkl', 'rb') as f:
        y = pickle.load(f)

CPU times: user 24 ms, sys: 1.29 s, total: 1.32 s
Wall time: 1.32 s


NMF 가 차원을 축소한 대상은 Bag of Words 로 표현된 문서벡터 입니다. 9,774 차원으로 표현되는 문서가 100 차원으로 표현됩니다.

In [4]:
y.shape

(30091, 100)

9,774 개의 단어 역시 100 차원으로 표현됩니다. SVD 처럼 components_ 에 정보가 저장되어 있습니다.

이를 반대로 해석하면 100 개의 components 들이 각각 9,774 개의 term weight vector 로 표현된 것과 같습니다. 30091 개의 문서를 표현할 수 있는 100 개의 components 입니다. 이는 마치 topic vector 의 역할을 합니다.

In [5]:
nmf.components_.shape

(100, 9774)

components vector 를 topic_vector 로 복사해둡니다.

In [6]:
topic_vector = nmf.components_.copy()

한 component 에 대하여 weight 의 크기가 큰 순서대로 component keyword 를 선택할 수 있습니다. 일종의 labeling 입니다.

71 번 component 는 아이돌 방송 관련 토픽들 입니다.

In [7]:
def most_important_terms(topic_idx, topn=10):
    
    # sort by weight in decreasing order
    term_idxs = topic_vector[topic_idx,:].argsort()[::-1]

    # select top n terms
    if topn > 0:
        term_idxs = term_idxs[:topn]

    # form of [(idx, weight), ... ]
    weights = topic_vector[topic_idx, term_idxs]
    term_and_weights = [(t,w) for t,w in zip(term_idxs, weights)]

    # decode
    term_and_weights = [(idx_to_vocab[t], w) for t,w in term_and_weights]

    return term_and_weights

most_important_terms(71)

[('불독', 16.011034310171343),
 ('데뷔', 7.627838309186585),
 ('걸그룹', 6.566420812556478),
 ('쇼케이스', 4.602740865694784),
 ('키미', 4.3269350387987675),
 ('형은', 4.119217675175327),
 ('무대', 3.9953000852499865),
 ('소라', 3.451016450726792),
 ('롤링', 3.2674144688507427),
 ('세이', 3.146422564026698)]

1 번 components 는 미국 대선 관련 성분입니다.

In [8]:
most_important_terms(1)

[('트럼프', 24.060825373206004),
 ('클린턴', 16.330708437344487),
 ('토론', 10.376507974552318),
 ('대선', 7.041041755201666),
 ('후보', 5.324686093570161),
 ('힐러리', 4.617175691684361),
 ('주장', 4.14022124957622),
 ('공화당', 3.4868372768982976),
 ('3차', 3.4362118631731096),
 ('선거', 2.988377886302627)]

한 단어에 대하여 weight 가 큰 components 를 찾을 수도 있습니다.

In [9]:
def most_relavant_topics(term, n_topics=5, n_terms=20):
    term_idx = vocab_to_idx.get(term, -1)
    if term_idx < 0:
        return []

    # slice topic weight vector
    relavant_topics = topic_vector[:,term_idx].argsort()[::-1]

    # select top topics
    if n_topics > 0:
        relavant_topics = relavant_topics[:n_topics]

    # select important terms for each topic (component)
    topic_terms = []
    for topic_idx in relavant_topics:
        # idx, score, terms
        topic_terms.append(
            (topic_idx,
             topic_vector[topic_idx, term_idx],
             most_important_terms(topic_idx, n_terms))
        )
    
    return topic_terms

단어의 빈도수를 확인하기 위하여 term frequency matrix 에 row sum 을 하여 frequency vector 를 만듭니다.

In [10]:
import numpy as np

term_frequency = np.asarray(x.sum(axis=0)).reshape(-1)
term_frequency.shape

(9774,)

`아이오아이` 라는 단어의 weight 가 큰 components 들의 top weighted words 입니다. `아이오아이` 라는 단어와 관련된 토픽 (components) 은 크게 3 개 정도로 학습되었지만, 해당 component 들에서의 weight 는 크지 않습니다.

In [11]:
query = '아이오아이'
query_idx = vocab_to_idx[query]
print('query term = {}, frequency = {}\n'.format(query, term_frequency[query_idx]))

relevant_topics = most_relavant_topics(query)

for topic_id, relavant_score, important_terms in relevant_topics:
    terms = ' '.join([term for term, _ in important_terms])
    print('component#{} ({:.4}): {}'.format(topic_id, relavant_score, terms))

query term = 아이오아이, frequency = 270

component#71 (1.406): 불독 데뷔 걸그룹 쇼케이스 키미 형은 무대 소라 롤링 세이 마포구 출연 매력 오전 멤버들 101 싱글 프로듀스 강렬 표현
component#11 (0.9662): 공개 모습 드라마 촬영 배우 방송 캐릭터 연기 도깨비 공유 출연 선보 매력 화보 한편 현장 연출 변신 기대 특히
component#79 (0.2516): 트와이스 1위 기록 스트리밍 방탄소년단 차트 누적 발표 뮤직비디오 올해 가온차트 2016년 데뷔 유튜브 최고 공개 미니앨범 조회수 20일 차지
component#78 (0.04374): 신화 앨범 발매 13집 팬들 정규 컴백 공개 활동 11월 이번 그룹 콘서트 데뷔 예정 멤버들 곡들 19년 기대 29일
component#34 (0.02382): 사진 이미지 역사 가치 인스타그램 글과 제공 자료 제주 예술 마이데일리 기사 모습 의미 카메라 평가 가운데 자신 대상 일어


`트와이스` 가 해당 시기에 더 자주 등장한 단어였기 때문에 weight 가 훨씬 크게 학습되었습니다. component#79 는 아이돌 관련 component 임을 알 수 있습니다.

In [12]:
query = '트와이스'
query_idx = vocab_to_idx[query]
print('query term = {}, frequency = {}\n'.format(query, term_frequency[query_idx]))

relevant_topics = most_relavant_topics(query)

for topic_id, relavant_score, important_terms in relevant_topics:
    terms = ' '.join([term for term, _ in important_terms])
    print('component#{} ({:.4}): {}'.format(topic_id, relavant_score, terms))

query term = 트와이스, frequency = 655

component#79 (5.795): 트와이스 1위 기록 스트리밍 방탄소년단 차트 누적 발표 뮤직비디오 올해 가온차트 2016년 데뷔 유튜브 최고 공개 미니앨범 조회수 20일 차지
component#19 (0.1202): 네이버 의장 서비스 이사회 유럽 한성숙 이해진 글로벌 부사장 3월 내년 대표이사 김상헌 대표 라인 내정자 의장직 업계 내정 인터넷
component#13 (0.01512): 차태현 김유정 영화 박보검 서현진 연기 사랑 때문 성동일 김윤혜 박근형 구르미 달빛 주지홍 호흡 많이 출연 배우 드라마 기억
component#99 (0.0): 모바일 삼성전자 10나노 기술 출시 제공 메모리 기존 용량 기기 글로벌 고객들 스마트폰 제품 세계 2배 공급 16 솔루션 교환
component#25 (0.0): 디자이너 브랜드 마이데일리 진행 2017 동대문디자인플라자 기사 유진 헤라서울패션위크 참석 컬렉션 22일 실시간 보도자료 구매 뉴미디어 신진 제보 참가 6개


'트럼프'라는 단어의 weight 가 큰 components 들의 top weighted words 입니다. component#1 과 component#39 에 집중되어 있으며, term frequency 가 크기 때문에 weight 도 큽니다.

In [13]:
query = '트럼프'
query_idx = vocab_to_idx[query]
print('query term = {}, frequency = {}\n'.format(query, term_frequency[query_idx]))

relevant_topics = most_relavant_topics(query)

for topic_id, relavant_score, important_terms in relevant_topics:
    terms = ' '.join([term for term, _ in important_terms])
    print('component#{} ({:.4}): {}'.format(topic_id, relavant_score, terms))

query term = 트럼프, frequency = 3540

component#1 (24.06): 트럼프 클린턴 토론 대선 후보 힐러리 주장 공화당 3차 선거 민주당 도널드 대선후보 미국 결과 라스베이거스 이날 지지 발언 불복
component#39 (10.0): 후보 지금 힐러리 말씀 있습니다 트럼프 이런 때문 제가 사실 클린턴 미국 생각합니다 생각 발언 경제 대통령 우리 국가 도널드
component#57 (0.06952): 여성 남성 남성들 여성들 혐오 자신 생각 사회 임신 말하는 운동 표현 이상 남자 낙태 사건 차별 비율 여자 사용
component#14 (0.04523): 중국 필리핀 두테르테 양국 남중국해 베이징 주석 정상회담 분쟁 협력 영유권 시진핑 방문 성장률 투자 갈등 수출 달러 경제 로드
component#74 (0.04333): 한국 세계 국내 최초 국가 이번 우리나라 영국 아시아 주제 문화재 해외 프랑스 독일 선생 태국 기증 독립 처음 언어


NMF 는 SVD 와 활용하는 방법은 비슷합니다. component 를 학습하는 방식이 다릅니다.